In [ ]:
!pip install -q gym numpy gymnasium stable-baselines3

In [ ]:
import carla
import os
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from time import sleep
import queue

In [ ]:
class CarlaEnv(gym.Env):

    def __init__(self, params):
        # Initial parameters
        self.desired_speed = params['desired_speed']
        self.number_of_vehicles = params['number_of_vehicles']
        self.number_of_walkers = params['number_of_walkers']
        self.max_ego_spawn_times = params['max_ego_spawn_times']
        # self.camera_width = params['camera_width']
        # self.camera_height = params['camera_height']
        self.camera_width = 800
        self.camera_height = 600

        # Destination
        self.dests = None

        # action space
        # load in the the discrete action space for each type of output
        self.discrete_act = [params['discrete_acc'], params['discrete_steer'], params['discrete_brake']]
        self.n_acc = len(self.discrete_act[0])
        self.n_steer = len(self.discrete_act[1])
        self.n_brake = len(self.discrete_act[2])
        # Accel and steer are seperated as it makes no sense to use both at same time
        self.action_space = spaces.Discrete((self.n_acc*self.n_steer) + (self.n_steer*self.n_brake))

        # observation space
        # ss -> semantic segmentation
        # default camera size 600x800
        observation_space_dict = {
            'ss_camera_front': spaces.Box(low=0, high=12, shape=(self.camera_height, self.camera_width), dtype=np.uint8),
            # Add any other observation spaces here
        }
        self.observation_space = spaces.Dict(observation_space_dict)

        # Connect to carla server
        client_name = os.environ.get("CLIENT_NAME", "DOES NOT EXIST")
        if client_name == "DOES NOT EXIST":
            raise Exception("The environment variable for the container name of the carla server has not been set")
        # Connect to the client and retrieve the world object
        client = carla.Client(client_name, 2000)
        client.set_timeout(10.0)
        self.world = client.get_world()
        print('Carla server connected!')

        # Set weather
        self.world.set_weather(carla.WeatherParameters.ClearNoon)

        # Get spawn points
        self.vehicle_spawn_points = list(self.world.get_map().get_spawn_points())
        self.walker_spawn_points = []
        for i in range(self.number_of_walkers):
            spawn_point = carla.Transform()
            loc = self.world.get_random_location_from_navigation()
            if (loc != None):
                spawn_point.location = loc
                self.walker_spawn_points.append(spawn_point)

        # Create the ego vehicle blueprint
        self.ego_bp = self.world.get_blueprint_library().find('vehicle.mini.cooper_s_2021')

        # Collision sensor
        self.collision_hist = [] # The collision history
        self.collision_hist_l = 1 # collision history length
        self.collision_bp = self.world.get_blueprint_library().find('sensor.other.collision')

        # Camera sensor
        self.camera_img = np.zeros((self.camera_height, self.camera_width), dtype=np.uint8)
        self.camera_trans = carla.Transform(carla.Location(x=0.5, z=2.0))
        self.camera_bp = self.world.get_blueprint_library().find('sensor.camera.rgb')
        # Modify the attributes of the blueprint to set image resolution and field of view.
        self.camera_bp.set_attribute('image_size_x', str(self.camera_width))
        self.camera_bp.set_attribute('image_size_y', str(self.camera_height))

        # Set synchronous mode settings
        self.settings = self.world.get_settings()
        self.settings.fixed_delta_seconds = 0.4 # Most stable value

        # Record the time of total steps and resetting steps
        self.reset_step = 0
        self.total_step = 0
    
    def reset(self):
        # Clear sensor objects
        self.camera_sensor = None
        self.collision_sensor = None

        # Delete sensors, vehicles and walkers
        self._clear_all_actors(['sensor.other.collision', 'sensor.lidar.ray_cast', 'sensor.camera.rgb', 'vehicle.*', 'controller.ai.walker', 'walker.*'])

        # Disable sync mode
        self._set_synchronous_mode(False)

        # Spawn surrounding vehicles
        random.shuffle(self.vehicle_spawn_points)
        count = self.number_of_vehicles
        if count > 0:
        for spawn_point in self.vehicle_spawn_points:
            if self._try_spawn_random_vehicle_at(spawn_point, number_of_wheels=[4]):
            count -= 1
            if count <= 0:
            break
        while count > 0:
        if self._try_spawn_random_vehicle_at(random.choice(self.vehicle_spawn_points), number_of_wheels=[4]):
            count -= 1

        # Spawn pedestrians
        random.shuffle(self.walker_spawn_points)
        count = self.number_of_walkers
        if count > 0:
          for spawn_point in self.walker_spawn_points:
            if self._try_spawn_random_walker_at(spawn_point):
              count -= 1
            if count <= 0:
              break
        while count > 0:
          if self._try_spawn_random_walker_at(random.choice(self.walker_spawn_points)):
            count -= 1
        # Get actors polygon list
        self.vehicle_polygons = []
        vehicle_poly_dict = self._get_actor_polygons('vehicle.*')
        self.vehicle_polygons.append(vehicle_poly_dict)
        self.walker_polygons = []
        walker_poly_dict = self._get_actor_polygons('walker.*')
        self.walker_polygons.append(walker_poly_dict)
        
        # Spawn the ego vehicle
        ego_spawn_times = 0
        while True:
            if ego_spawn_times > self.max_ego_spawn_times:
                self.reset()
            transform = random.choice(self.vehicle_spawn_points)
            if self._try_spawn_ego_vehicle_at(transform):
                break
            else:
                ego_spawn_times += 1
                time.sleep(0.1)
                
        # Add collision sensor
        self.collision_sensor = self.world.spawn_actor(self.collision_bp, carla.Transform(), attach_to=self.ego)
        self.collision_sensor.listen(lambda event: get_collision_hist(event))
        def get_collision_hist(event):
            impulse = event.normal_impulse
            intensity = np.sqrt(impulse.x**2 + impulse.y**2 + impulse.z**2)
            self.collision_hist.append(intensity)
            if len(self.collision_hist)>self.collision_hist_l:
                self.collision_hist.pop(0)
        self.collision_hist = []

        # Add camera sensor
        self.camera_sensor = self.world.spawn_actor(self.camera_bp, self.camera_trans, attach_to=self.ego)
        self.camera_sensor.listen(lambda data: get_camera_img(data))
        def get_camera_img(data):
            array = np.frombuffer(data.raw_data, dtype = np.dtype("uint8"))
            array = np.reshape(array, (data.height, data.width, 4))
            array = array[:, :, 2]
            self.camera_img = array

        # Update timesteps
        self.time_step=0
        self.reset_step+=1
        
        # Enable sync mode
        self.settings.synchronous_mode = True
        self.world.apply_settings(self.settings)

        return self._get_obs()



        

        

    def _try_spawn_random_vehicle_at(self, transform, number_of_wheels=[4]):
        """Try to spawn a surrounding vehicle at specific transform with random bluprint.
    
        Args:
          transform: the carla transform object.
    
        Returns:
          Bool indicating whether the spawn is successful.
        """
        blueprint = self._create_vehicle_bluepprint('vehicle.*', number_of_wheels=number_of_wheels)
        blueprint.set_attribute('role_name', 'autopilot')
        vehicle = self.world.try_spawn_actor(blueprint, transform)
        if vehicle is not None:
            vehicle.set_autopilot()
            return True
        return False

    def _try_spawn_random_walker_at(self, transform):
        """Try to spawn a walker at specific transform with random bluprint.
    
        Args:
          transform: the carla transform object.
    
        Returns:
          Bool indicating whether the spawn is successful.
        """
        walker_bp = random.choice(self.world.get_blueprint_library().filter('walker.*'))
        # set as not invencible
        if walker_bp.has_attribute('is_invincible'):
            walker_bp.set_attribute('is_invincible', 'false')
        walker_actor = self.world.try_spawn_actor(walker_bp, transform)
    
        if walker_actor is not None:
            walker_controller_bp = self.world.get_blueprint_library().find('controller.ai.walker')
            walker_controller_actor = self.world.spawn_actor(walker_controller_bp, carla.Transform(), walker_actor)
            # start walker
            walker_controller_actor.start()
            # set walk to random point
            walker_controller_actor.go_to_location(self.world.get_random_location_from_navigation())
            # random max speed
            walker_controller_actor.set_max_speed(1 + random.random())    # max speed between 1 and 2 (default is 1.4 m/s)
            return True
        return False
    
    def _clear_all_actors(self, actor_filters):
        """Clear specific actors."""
        for actor_filter in actor_filters:
            for actor in self.world.get_actors().filter(actor_filter):
                if actor.is_alive:
                if actor.type_id == 'controller.ai.walker':
                    actor.stop()
                actor.destroy()
    
    def _set_synchronous_mode(self, synchronous = True):
        """Set whether to use the synchronous mode.
        """
        self.settings.synchronous_mode = synchronous
        self.world.apply_settings(self.settings)






        
        
        